In [25]:
from pathlib import Path
import kagglehub

# Download latest version
path = kagglehub.dataset_download("changheonkim/iam-trocr")
path = Path(path)/"IAM"
print("Path to dataset files:", path)

Using Colab cache for faster access to the 'iam-trocr' dataset.
Path to dataset files: /kaggle/input/iam-trocr/IAM


In [26]:
import os

# Assuming 'path' variable holds the base directory from kagglehub.dataset_download
# If not, please replace 'path' with the correct directory string, e.g., '/content/IAM'
if 'path' in globals():
    print(f"Listing directories in: {path}")
    # Use a shell command to list only directories recursively, and sort them
    !ls {path/"image"}


Listing directories in: /kaggle/input/iam-trocr/IAM
c04-110-00.jpg	e06-070-02.jpg	 g07-000b-00.jpg  n02-157-05.jpg
c04-110-01.jpg	e06-070-03.jpg	 g07-000b-01.jpg  n02-157-06.jpg
c04-110-02.jpg	e06-070-04.jpg	 g07-000b-02.jpg  n02-157-07.jpg
c04-110-03.jpg	e06-070-05.jpg	 g07-000b-03.jpg  n02-157-08.jpg
c04-116-00.jpg	e06-070-06.jpg	 g07-000b-04.jpg  n03-038-00.jpg
c04-116-01.jpg	e06-070-07.jpg	 g07-000b-05.jpg  n03-038-01.jpg
c04-116-02.jpg	e06-070-08.jpg	 g07-000b-06.jpg  n03-038-02.jpg
c04-116-03.jpg	e06-070-09.jpg	 g07-000b-07.jpg  n03-038-03.jpg
c04-134-00.jpg	f04-032-00.jpg	 g07-000b-08.jpg  n03-038-04.jpg
c04-134-01.jpg	f04-032-01.jpg	 g07-000b-09.jpg  n03-038-05.jpg
c04-134-02.jpg	f04-032-02.jpg	 g07-079a-00.jpg  n03-038-06.jpg
c04-134-03.jpg	f04-032-03.jpg	 g07-079a-01.jpg  n03-064-00.jpg
c04-134-04.jpg	f04-032-04.jpg	 g07-079a-02.jpg  n03-064-01.jpg
c04-134-05.jpg	f04-032-05.jpg	 g07-079a-03.jpg  n03-064-02.jpg
c04-134-06.jpg	f04-032-06.jpg	 g07-079a-04.jpg  n03-064-03.jpg
c04

In [27]:
import glob

# Assuming 'path' is defined and points to the base directory of the dataset
# The images are located in the 'image' subdirectory relative to 'path'
image_directory = path / "image"

# Use glob to find all .jpg files in the image directory
image_paths = sorted(glob.glob(str(image_directory / "*.jpg")))

print(f"Found {len(image_paths)} images in the dataset.")
print("First 5 image paths:")
for i, img_path in enumerate(image_paths[:5]):
    print(f"  {i+1}: {img_path}")

Found 2915 images in the dataset.
First 5 image paths:
  1: /kaggle/input/iam-trocr/IAM/image/c04-110-00.jpg
  2: /kaggle/input/iam-trocr/IAM/image/c04-110-01.jpg
  3: /kaggle/input/iam-trocr/IAM/image/c04-110-02.jpg
  4: /kaggle/input/iam-trocr/IAM/image/c04-110-03.jpg
  5: /kaggle/input/iam-trocr/IAM/image/c04-116-00.jpg


In [28]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms

class IAMImageDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB') # Load as RGB (or 'L' for grayscale if preferred)

        if self.transform:
            image = self.transform(image)

        # For now, we'll just return the image.
        # In a real scenario, you'd also load and return the corresponding label/text.
        return image

# Define transformations (you can customize these)
# Example: Resize to 224x224 and convert to tensor, then normalize
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # ImageNet normalization
])

# Instantiate your custom dataset
# 'image_paths' is assumed to be defined from a previous cell
iam_dataset = IAMImageDataset(image_paths=image_paths, transform=transform)

print(f"Number of samples in the dataset: {len(iam_dataset)}")

# To get a single image from the dataset:
single_image_tensor = iam_dataset[0]
print(f"Shape of a single image tensor: {single_image_tensor.shape}")

Number of samples in the dataset: 2915
Shape of a single image tensor: torch.Size([3, 224, 224])


In [29]:
from transformers import VisionEncoderDecoderModel, TrOCRProcessor
import torch

# Load the model
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/478 [00:00<?, ?it/s]

VisionEncoderDecoderModel LOAD REPORT from: microsoft/trocr-base-handwritten
Key                         | Status  | 
----------------------------+---------+-
encoder.pooler.dense.weight | MISSING | 
encoder.pooler.dense.bias   | MISSING | 

Notes:
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=False)
              (key): Linear(in_features=768, out_features=768, bias=False)
              (value): Linear(in_features=768, out_features=768, bias=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (i

In [30]:
len(model.decoder.model.decoder.layers)

12

In [44]:
print(model.decoder)

TrOCRForCausalLM(
  (model): TrOCRDecoderWrapper(
    (decoder): TrOCRDecoder(
      (embed_tokens): TrOCRScaledWordEmbedding(50265, 1024, padding_idx=1)
      (embed_positions): TrOCRLearnedPositionalEmbedding(514, 1024)
      (layernorm_embedding): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-11): 12 x TrOCRDecoderLayer(
          (self_attn): TrOCRAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (activation_fn): GELUActivation()
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (encoder_attn): TrOCRAttention(
            (k_proj): Linear(in_features=768, out_features=1024, bias=True)
            (v

In [33]:
for i, layer in enumerate(model.decoder.model.decoder.layers):
    print(f"Camada {i} ->", type(layer))

Camada 0 -> <class 'transformers.models.trocr.modeling_trocr.TrOCRDecoderLayer'>
Camada 1 -> <class 'transformers.models.trocr.modeling_trocr.TrOCRDecoderLayer'>
Camada 2 -> <class 'transformers.models.trocr.modeling_trocr.TrOCRDecoderLayer'>
Camada 3 -> <class 'transformers.models.trocr.modeling_trocr.TrOCRDecoderLayer'>
Camada 4 -> <class 'transformers.models.trocr.modeling_trocr.TrOCRDecoderLayer'>
Camada 5 -> <class 'transformers.models.trocr.modeling_trocr.TrOCRDecoderLayer'>
Camada 6 -> <class 'transformers.models.trocr.modeling_trocr.TrOCRDecoderLayer'>
Camada 7 -> <class 'transformers.models.trocr.modeling_trocr.TrOCRDecoderLayer'>
Camada 8 -> <class 'transformers.models.trocr.modeling_trocr.TrOCRDecoderLayer'>
Camada 9 -> <class 'transformers.models.trocr.modeling_trocr.TrOCRDecoderLayer'>
Camada 10 -> <class 'transformers.models.trocr.modeling_trocr.TrOCRDecoderLayer'>
Camada 11 -> <class 'transformers.models.trocr.modeling_trocr.TrOCRDecoderLayer'>


In [37]:
for name, module in model.decoder.named_modules():
    print(name, "->", type(module))

 -> <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'>
model -> <class 'transformers.models.trocr.modeling_trocr.TrOCRDecoderWrapper'>
model.decoder -> <class 'transformers.models.trocr.modeling_trocr.TrOCRDecoder'>
model.decoder.embed_tokens -> <class 'transformers.models.trocr.modeling_trocr.TrOCRScaledWordEmbedding'>
model.decoder.embed_positions -> <class 'transformers.models.trocr.modeling_trocr.TrOCRLearnedPositionalEmbedding'>
model.decoder.layernorm_embedding -> <class 'torch.nn.modules.normalization.LayerNorm'>
model.decoder.layers -> <class 'torch.nn.modules.container.ModuleList'>
model.decoder.layers.0 -> <class 'transformers.models.trocr.modeling_trocr.TrOCRDecoderLayer'>
model.decoder.layers.0.self_attn -> <class 'transformers.models.trocr.modeling_trocr.TrOCRAttention'>
model.decoder.layers.0.self_attn.k_proj -> <class 'torch.nn.modules.linear.Linear'>
model.decoder.layers.0.self_attn.v_proj -> <class 'torch.nn.modules.linear.Linear'>
model.decoder.layers

In [43]:
total_params = sum(p.numel() for p in model.parameters())
print("Total parameters:", total_params)

Total parameters: 333921792


In [36]:
for name, param in model.decoder.named_parameters():
    print(name, param.shape)

model.decoder.embed_tokens.weight torch.Size([50265, 1024])
model.decoder.embed_positions.weight torch.Size([514, 1024])
model.decoder.layernorm_embedding.weight torch.Size([1024])
model.decoder.layernorm_embedding.bias torch.Size([1024])
model.decoder.layers.0.self_attn.k_proj.weight torch.Size([1024, 1024])
model.decoder.layers.0.self_attn.k_proj.bias torch.Size([1024])
model.decoder.layers.0.self_attn.v_proj.weight torch.Size([1024, 1024])
model.decoder.layers.0.self_attn.v_proj.bias torch.Size([1024])
model.decoder.layers.0.self_attn.q_proj.weight torch.Size([1024, 1024])
model.decoder.layers.0.self_attn.q_proj.bias torch.Size([1024])
model.decoder.layers.0.self_attn.out_proj.weight torch.Size([1024, 1024])
model.decoder.layers.0.self_attn.out_proj.bias torch.Size([1024])
model.decoder.layers.0.self_attn_layer_norm.weight torch.Size([1024])
model.decoder.layers.0.self_attn_layer_norm.bias torch.Size([1024])
model.decoder.layers.0.encoder_attn.k_proj.weight torch.Size([1024, 768])
m

In [40]:
import inspect
from transformers.models.trocr.modeling_trocr import TrOCRDecoderLayer

print(inspect.getsource(TrOCRDecoderLayer))

class TrOCRDecoderLayer(GradientCheckpointingLayer):
    def __init__(self, config: TrOCRConfig, layer_idx=None):
        super().__init__()
        self.embed_dim = config.hidden_size

        self.self_attn = TrOCRAttention(
            config,
            embed_dim=self.embed_dim,
            num_heads=config.decoder_attention_heads,
            dropout=config.attention_dropout,
            is_decoder=True,
            layer_idx=layer_idx,
        )
        self.dropout = config.dropout
        self.activation_fn = ACT2FN[config.activation_function]
        self.activation_dropout = config.activation_dropout

        self.self_attn_layer_norm = nn.LayerNorm(self.embed_dim)

        if config.is_decoder:
            self.encoder_attn = TrOCRAttention(
                config,
                embed_dim=self.embed_dim,
                num_heads=config.decoder_attention_heads,
                kdim=config.cross_attention_hidden_size,
                vdim=config.cross_attention_hidden_size